In [100]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import random 

In [101]:
# Carrega o data set
data = load_iris()

# Atribui valores à entrada e à saída 
X = data.data
y = data.target

In [102]:
# Separando dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Dados para treinamento: {len(X_train)}\nDados para teste: {len(X_test)}')

Dados para treinamento: 120
Dados para teste: 30


### PARÂMETROS GLOBAIS

In [103]:
CLASS_NUM = 3
POPULATION_SIZE = 60
SELECTION_RATE = 0.2 # 20%
MUTATION_RATE = 0.1
MUTATION_DEVIATION = 0.1
REPLACEMENT_RATE = 0.1
NUM_OF_GENERATIONS = 50


### GERAÇÃO DA POPULAÇÃO DE ANTICORPOS

In [104]:
# Classe Anticorpo

class Antibody:
    def __init__(self, vector, label):
        self.vector = vector
        self.label = label
        self.affinity = None

In [105]:
# Classe Antígeno

class Antigen:
    def __init__(self, vector, label):
        self.vector = vector
        self.label = label

Para a geração da população, iremos escolher aleatoriamente os valores do vetor, dentro de um limite estimado pelo menor e maior valor de cada coluna do dataset.

Por fim, iremos atribuir igualmente as labels para cada anticorpo,de modo que cada classe tenha a mesma quantidade de anticorpos gerados.

In [106]:
# Extraindo o valor minimo e máximo de cada atributo da Iris

min_sepal_length = min(X_train[:, 0])
max_sepal_length = max(X_train[:, 0])

min_sepal_width = min(X_train[:, 1])
max_sepal_width = max(X_train[:, 1])

min_petal_length = min(X_train[:, 2])
max_petal_length = max(X_train[:, 2])

min_petal_width = min(X_train[:, 3])
max_petal_width = max(X_train[:, 3])

# Salvando os limites inferiores e superiores em uma lista
min_bounds = [min_sepal_length, min_sepal_width, min_petal_length, min_petal_width]
max_bounds = [max_sepal_length, max_sepal_width, max_petal_length, max_petal_width]

print(f'Limite do comprimento da sépala: [{min_sepal_length}, {max_sepal_length}]\nLimite da largura da sépala: [{min_sepal_width}, {max_sepal_width}]\nLimite do comprimento da pétala: [{min_petal_length}, {max_petal_length}]\nLimite da largura da sépala: [{min_petal_width}, {max_petal_width}]\n')

Limite do comprimento da sépala: [4.3, 7.7]
Limite da largura da sépala: [2.0, 4.4]
Limite do comprimento da pétala: [1.0, 6.7]
Limite da largura da sépala: [0.1, 2.5]



In [107]:
# Função para gerar um anticorpo

def generateAntibody(label):

    # Escolhe um valor aleatorio dentro da faixa estimada
    vector = [
        random.uniform(min_sepal_length, max_sepal_length),
        random.uniform(min_sepal_width, max_sepal_width),
        random.uniform(min_petal_length, max_sepal_length),
        random.uniform(min_petal_width, max_petal_width)
    ]
    
    antibody = Antibody(vector, label)
    return antibody

In [108]:
# Função para criar a população com base no seu tamanho e numero de classes

def generatePopulation():

    population = []
    # Cria labels igualmente com base no numero de classes
    labels = [i % CLASS_NUM for i in range(POPULATION_SIZE)]

    for i in range(POPULATION_SIZE):
        population.append(generateAntibody(labels[i]))
    
    return population

### Avaliação do Fitness

In [109]:
# Função para a avaliação do fitness em relação ao antígeno
# O maior fitness possível será igual a 1

def evaluateFitness(antibody, antigen):
    # Caso possuam a mesma label
    if antibody.label == antigen.label:
    
        """
        Utiliza-se o calcúlo do inverso da distância Euclidiana
        para avaliar o fitness. Logo

        Nota = 1/Distancia(anticorpo, antigeno)

        quanto menor a distancia, maior será  nota
        """
        # Método do NumPy que calcula a distância euclidiana entre dois pontos
        distance = np.linalg.norm(np.array(antibody.vector) - np.array(antigen.vector))

        return 1/(1 + distance) # Operação para evitar distância com zero
    # Significa que o anticorpo eh um falso positivo   
    else: 
        return 0


In [110]:
# Célula para realização de teste
pop = generatePopulation() # Gera uma população
antigen = Antigen(X_train[0], y_train[0]) # Pega o primeiro antigeno

# Realiza a avaliação de cada anticorpo da população
print(f'Para o Antígeno {antigen.label} de vetor {antigen.vector}\n')
for antibody in pop:
    print(f'Anticorpo {antibody.label}, {antibody.vector} : {evaluateFitness(antibody, antigen)}\n')

Para o Antígeno 0 de vetor [4.6 3.6 1.  0.2]

Anticorpo 0, [5.066573992241127, 3.5048397279145105, 3.40883066778062, 0.8146593948793722] : 0.2831891425911716

Anticorpo 1, [5.1353510895897925, 4.048614438210353, 1.4382970499736132, 1.2099695054102275] : 0

Anticorpo 2, [5.807982736172033, 4.255772671029751, 2.5869280385449773, 1.1945419149591727] : 0

Anticorpo 0, [4.983460143706609, 2.190361428849049, 4.894276463186172, 1.2456056670934141] : 0.18908298643369473

Anticorpo 1, [7.080476506340097, 3.0019129632606276, 2.8991808629787066, 2.192980521489485] : 0

Anticorpo 2, [6.791481352321126, 4.213920518238881, 4.843198394795452, 2.3472423169138077] : 0

Anticorpo 0, [7.460164997006229, 3.7994365959380745, 5.721085102702553, 1.3625051882235233] : 0.1505004111163086

Anticorpo 1, [6.920031310313886, 3.065124905354637, 2.8681201706511845, 2.0951189005960327] : 0

Anticorpo 2, [6.052772696414045, 2.5573797064851203, 3.7101199922148953, 1.220013639367607] : 0

Anticorpo 0, [5.834242194234547

### Seleção dos N melhores anticorpos

In [111]:
# Função para realizar a seleção

def selectBestFitness(evaluated_antibodies):
    # Ordena a lista do menor para o maior fitness
    ordered_list = sorted(evaluated_antibodies, key=lambda x : x[0], reverse=True)

    # Determina a quantidade de melhores com base no SELECTION RATE
    best_qtd = int(POPULATION_SIZE * SELECTION_RATE)
    # Seleciona essa quantidade na lista
    bests = ordered_list[:best_qtd]
    return bests # Retorna a parcela da lista selectionada

In [112]:
# Célula para realização de teste

# Lista com os anticorpos e seus respectivos fitness
evaluated_antibodies = []

for antibody in pop:
    evaluated_antibody = []
    # Avalia o fitness de cada anticorpo
    evaluated_antibody.append(evaluateFitness(antibody, antigen))
    evaluated_antibody.append(antibody)
    # Adiciona a lista com o anticorpo e seu fitness
    evaluated_antibodies.append(evaluated_antibody)

# Resgata os N melhores fitness
bests = selectBestFitness(evaluated_antibodies)

#Imprime
for i ,antibody in enumerate(bests):
    print(f'Anticorpo {i + 1} : {antibody[0]}')

Anticorpo 1 : 0.36975079599360167
Anticorpo 2 : 0.34747981053794064
Anticorpo 3 : 0.34122272730741116
Anticorpo 4 : 0.3170109400536747
Anticorpo 5 : 0.30001473368945564
Anticorpo 6 : 0.2831891425911716
Anticorpo 7 : 0.24455464679490538
Anticorpo 8 : 0.2412928552995048
Anticorpo 9 : 0.2238498491979606
Anticorpo 10 : 0.2187864393882793
Anticorpo 11 : 0.21854909844228612
Anticorpo 12 : 0.19222576591995885


### Clonagem dos anticorpos mais promissores

Para a etapa de clonagem, utilizamos o seguinte equação para definir quantos clones cada fitness irá receber:

$$
QC^i_k = \left( \frac{af^i_k}{\sum_{k=1}^{n} af^i_k} \right) \cdot Cl
$$

Onde:
- `QC^i_k`: Quantidade de clones gerados para o anticorpo *k* na iteração *i*
- `af^i_k`: Afinidade do anticorpo *k* na iteração *i*
- `Cl`: Quantidade total de clones a serem distribuídos
- `n`: Número total de anticorpos selecionados

In [124]:
# Função para realizar a clonagem

def clone(bests):

    # Lista final para armazenar os clone
    clones = []

    # Define o numero total de clone
    total_clones = POPULATION_SIZE - len(bests)

    # Somatorio de afinidades
    fitness = [x[0] for x in bests] # cria lista de afinidades
    fitness_sum = sum(fitness) # Soma total das afinidades

    # Lista temporária para armazenar as classes de anticorpos
    antibodies = [x[1] for x in bests] # cria uma lista de anticorpos

    # Calcula a quantidade de clones para cada fitness e armazena em uma lista
    qtd_clones = [round((c/fitness_sum) * total_clones) for c in fitness]

    # Realiza a clonagem ideal para cada fitness e armazena na lista final de clones
    counter = 0
    while counter < len(qtd_clones):
        for i in range(qtd_clones[counter]):
            # Lista que armazena a classe do anticorpo e seu fitness
            pair = []
            pair.append(fitness[counter])
            pair.append(antibodies[counter])
            # Salva na lista final
            clones.append(pair)
        counter += 1
    
    # Retorna o total de clones
    return clones

In [114]:
# Célula para a realização de teste
clones = clone(bests)
counter = 0

for i, clone in enumerate(clones):
    print(f'Clone {i + 1}: {clone[0]}')
    counter+= 1
    

print(f'Total de clones gerados: {counter}')

Clone 1: 0.36975079599360167
Clone 2: 0.36975079599360167
Clone 3: 0.36975079599360167
Clone 4: 0.36975079599360167
Clone 5: 0.36975079599360167
Clone 6: 0.34747981053794064
Clone 7: 0.34747981053794064
Clone 8: 0.34747981053794064
Clone 9: 0.34747981053794064
Clone 10: 0.34747981053794064
Clone 11: 0.34122272730741116
Clone 12: 0.34122272730741116
Clone 13: 0.34122272730741116
Clone 14: 0.34122272730741116
Clone 15: 0.34122272730741116
Clone 16: 0.3170109400536747
Clone 17: 0.3170109400536747
Clone 18: 0.3170109400536747
Clone 19: 0.3170109400536747
Clone 20: 0.3170109400536747
Clone 21: 0.30001473368945564
Clone 22: 0.30001473368945564
Clone 23: 0.30001473368945564
Clone 24: 0.30001473368945564
Clone 25: 0.2831891425911716
Clone 26: 0.2831891425911716
Clone 27: 0.2831891425911716
Clone 28: 0.2831891425911716
Clone 29: 0.24455464679490538
Clone 30: 0.24455464679490538
Clone 31: 0.24455464679490538
Clone 32: 0.24455464679490538
Clone 33: 0.2412928552995048
Clone 34: 0.2412928552995048


### Hipermutação dos anticorpos de acordo com a afinidade

Para a hipermutação, utilizamos a seguinte equação que é inversamente proporcional ao fitness dos clones:

$$
hipAnt^i_k = \left( 1 - \left( \frac{afin^i_k}{121} \right) \right) \cdot \beta
$$

Where:

- ` hipAnt^i_k `: taxa de hipermitação do anticorpo `k` na iteração `i`  
- ` afin^i_k `: afinidade do anticorpo `k` na iteração`i`  
- ` \beta `: fator de mutação

De forma a alterar o vetor 4D de cada clone, iremos apicar o a **Mutação** **Gaussiana**, a qual adiciona ou remove um valor aleatório de cada posição do vetor com base no MUTATION_DEVIATION

In [115]:
# Função que realizará a mutação dos clones e retornará os clones mutados, porém
# sem o fitness, o qual será recalculado na próxima etapa

def mutate(clones):
    mutated_clones = []
    # Seleciona todos os anticorpos presentes nos clone para serem mutados
    antibodies = [x[1] for x in clones]

    for i, antibody in enumerate(antibodies):
        afinity = clones[i][0] # Recupera a afinidade inicial do clone

        # Calcula a taxa de mutação para o clone especifico com base em sua afinidade
        hip_mutation_rate = (1 - (afinity/121)) * MUTATION_RATE

        # Inicializa lista que armazenará o vetor 4D mutado
        mutated_vector = []

        # Para cada valor no vetor 4D do clone, escolheremos aleatoriamente um valor
        # de mutação caso tenha acertado a probabilidade
        for j in range(len(antibody.vector)):
            # Gera um número aleatório entre 0 e 1
            rand = random.random()
            if rand <= hip_mutation_rate: # Irá realizar a mutação
                # Aplica a mutação Gaussiana
                value = antibody.vector[j] + np.random.normal(0, MUTATION_DEVIATION) 
                # Clipa o valor nos limites inferiores e superiores
                value = np.clip(value, min_bounds[j], max_bounds[j])
            else:
                value = antibody.vector[j]
            # Adiciona valor mutado    
            mutated_vector.append(value)

        # Cria um novo anticorpo com base na mutação do clone
        mutated_clones.append(Antibody(mutated_vector, antibody.label))
    # Retorna uma lista com os novos objetos originários da mutação dos clones
    return mutated_clones
        


In [116]:
# Célula para realização de teste

mutated_clones = mutate(clones)

for i in range(len(mutated_clones)):
    print(f'Clone original: {clones[i][1].vector}\nClone mutado  : {mutated_clones[i].vector}\n')

Clone original: [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Clone mutado  : [6.173160668459816, 2.854293655927052, 1.0864705153529683, 0.11433366347414094]

Clone original: [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Clone mutado  : [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]

Clone original: [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Clone mutado  : [6.173160668459816, 2.872201609236902, 1.0864705153529683, 0.11433366347414094]

Clone original: [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Clone mutado  : [6.225446349485865, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]

Clone original: [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Clone mutado  : [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]

Clone original: [5.75867574

### Metadinâmica

Nessa etapa, iremos juntar os clones mutados com a classe dos melhores fitness, que foram escolhidos anteriormente.
Para aumentar a diversidade e evitar que os valores não fiquem presos à um ótimo local, selecionaremos os N piores anticorpos e iremos substituí-los por N novos anticorpos gerados aleatoriamente

In [ ]:
def replacement(bests, mutated_clones, antigen):

    # Avalia os clone mutados e salva em uma lista
    evaluated_mutated = [[evaluateFitness(clone, antigen), clone] for clone in mutated_clones]

    # Combina as duas listas para serem ordenadas
    combined_pool = bests + evaluated_mutated
    # Ordena a lista por valor de fitness em ordem decrescente
    ordered_pool = sorted(combined_pool, key=lambda x : x[0], reverse=True)

    # Calcula a quantidade de anticorpos para serem excluidos
    replacement_num = int(POPULATION_SIZE * REPLACEMENT_RATE)

    # Exclui a quantidade calculada
    list_to_keep = ordered_pool[:POPULATION_SIZE - replacement_num]
    # Extrai todos os anticorpos da lista mantida
    antibodies = [x[1] for x in list_to_keep]

    # Gera novos anticorpos aleatorios
    new_population = []
    # Cria labels igualmente com base no numero de classes
    labels = [i % CLASS_NUM for i in range(replacement_num)]

    for i in range(replacement_num):
        new_population.append(generateAntibody(labels[i]))

    return antibodies + new_population


In [118]:
# Célula para realização de teste

new_generation = replacement(bests, mutated_clones, antigen)

for i, antibody in enumerate(new_generation):
    print(f'Anticorpo {i + 1} : {antibody.label}, {antibody.vector}')

Anticorpo 1 : 0, [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Anticorpo 2 : 0, [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Anticorpo 3 : 0, [6.173160668459816, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Anticorpo 4 : 0, [6.173160668459816, 2.872201609236902, 1.0864705153529683, 0.11433366347414094]
Anticorpo 5 : 0, [6.173160668459816, 2.854293655927052, 1.0864705153529683, 0.11433366347414094]
Anticorpo 6 : 0, [6.225446349485865, 2.9552111879949368, 1.0864705153529683, 0.11433366347414094]
Anticorpo 7 : 0, [5.758675747565107, 3.1202415817697413, 2.315297685189841, 0.6619499078854494]
Anticorpo 8 : 0, [5.758675747565107, 3.109651231473715, 2.315297685189841, 0.6619499078854494]
Anticorpo 9 : 0, [5.758675747565107, 3.109651231473715, 2.315297685189841, 0.6619499078854494]
Anticorpo 10 : 0, [5.758675747565107, 3.109651231473715, 2.315297685189841, 0.6619499078854494]
Anticorpo 11 : 0, [5.75867574756

### O ALGORITMO CLONAG

In [ ]:
def clonag(antigens):

    # Gera a população inicial
    population = generatePopulation()

    for antigen in antigens:
        for _ in range(NUM_OF_GENERATIONS):

            # PASSO 1 : AVALIAÇÃO 
            evaluated = []
            evaluated = [[evaluateFitness(antibody, antigen), antibody] for antibody in population]

            # PASSO 2 : SELECIONAR OS MELHORES
            bests = selectBestFitness(evaluated)

            # PASSO 3 : CLONAGEM
            clones = clone(bests)

            # PASSO 4: HIPERMUTAÇÃO
            mutated_clones = mutate(clones)

            # PASSO 5: METADINÂMICA
            population = replacement(bests, mutated_clones, antigen)

    # Retorna a população treinada       
    return population


### REALIZANDO O TREINAMENTO

In [137]:
# Preparando os antígenos de treinamento

train_antigens = []

for i in range(len(X_train)):
    train_antigens.append(Antigen(X_train[i], y_train[i]))


In [138]:
# Realizando o treino
trained_population = clonag(train_antigens)


In [139]:
for antibody in trained_population:
    print(f'{antibody.label}, {antibody.vector}')

0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [5.034787166012811, 3.455223376112184, 1.4516564753268104, 0.23050893539355197]
0, [